<a href="https://www.kaggle.com/code/emigiupponi/crypto-capitalization-apis?scriptVersionId=110038166" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Crypto Capitalization (Coinmetrics API)

# Setup

In [1]:
pip install coinmetrics-api-client

  Attempting uninstall: typer
    Found existing installation: typer 0.4.2
    Uninstalling typer-0.4.2:
      Successfully uninstalled typer-0.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.3.1 requires typer<0.5.0,>=0.3.0, but you have typer 0.6.1 which is incompatible.
spacy 3.3.1 requires typing-extensions<4.2.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.3.0 which is incompatible.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install coinmetrics-api-client -U

Note: you may need to restart the kernel to use updated packages.


In [3]:
from os import environ
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as image
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates
import plotly.express as px
import logging
from datetime import date, datetime, timedelta
from coinmetrics.api_client import CoinMetricsClient
import json
import logging
%matplotlib inline

In [4]:
# for plotting
# sns.set_theme()
# sns.set(rc={'figure.figsize':(12,8)})

In [5]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [6]:
# We recommend privately storing your API key in your local environment.
try:
    api_key = environ["CM_API_KEY"]
    logging.info("Using API key found in environment")
except KeyError:
    api_key = ""
    logging.info("API key not found. Using community client")

# Initialize the client
client = CoinMetricsClient(api_key)

In [7]:
# Get all assets that have a reference rate 
assets_refrate = client.catalog_metrics("CapMrktCurUSD")
# Get list of assets with daily ref rate 
# uncomment the top line to look at *every* asset with reference rates
# asset_with_ref_rates = assets_refrate[0]["frequencies"][0]["assets"]
asset_with_ref_rates = ['btc', 'eth', 'usdc']
#Query API for prices, daily CM reference rates as dataframe
metrics = "CapMrktCurUSD"
frequency = "1d"
start_time = "2011-11-08"
end_time = datetime.now().strftime('%Y-%m-%d')

logging.info("Getting prices...")
df_CapMrkt = client.get_asset_metrics(
    assets=asset_with_ref_rates,
    metrics=metrics,
    frequency=frequency,
    start_time=start_time,
    end_time=end_time
).to_dataframe()
# Assign datatypes
df_CapMrkt["time"] = pd.to_datetime(df_CapMrkt.time)
df_CapMrkt["CapMrktCurUSD"] = df_CapMrkt.CapMrktCurUSD.astype(float)

# Reshape dataset so assets are in columns, dates are the rows, and the values are prices
df_CapMrkt_pivot = df_CapMrkt.pivot(
    index="time",
    columns="asset",
    values="CapMrktCurUSD"
)

# Index each asset's time series to 1 
for col in df_CapMrkt_pivot.columns:
    logging.info(f"Calculating Reference rate for {col}....")
    # First price in time series
    first_price = df_CapMrkt_pivot[df_CapMrkt_pivot[col].notnull()][col].iloc[0]
    # Index time series
    df_CapMrkt_pivot[col] = df_CapMrkt_pivot[col]/first_price
    # Fill forward for Null values
    df_CapMrkt_pivot[col] = df_CapMrkt_pivot[col].ffill()

In [8]:
from matplotlib.pyplot import plot

In [9]:
import plotly.express as px
import pandas as pd

In [10]:
import plotly.graph_objects as go

In [11]:
# https://plotly.com/python/time-series/
fig = px.area(df_CapMrkt, x="time", y="CapMrktCurUSD", 
              title='Crypto Capitalization', 
              color="asset")
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

# Online

https://towardsdatascience.com/embedding-interactive-python-plots-on-the-web-84ceab57e417

In [12]:
pip install chart_studio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 257.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [13]:
import chart_studio
import plotly.express as px
import chart_studio.plotly as py

In [14]:
import chart_studio.plotly as py

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("CHARTSTUDIO_API_KEY")

chart_studio.tools.set_credentials_file(username='emigiupponi', 
                                        api_key=secret_value_0)


In [15]:
py.plot(fig, filename = 'cap', auto_open=True)

'https://plotly.com/~emigiupponi/27/'

In [16]:
secret_value_1 = user_secrets.get_secret("CHARTSTUDIO_BCRA_API_KEY")

In [17]:
chart_studio.tools.set_credentials_file(username='crypto-financial-stability-monitor', 
                                        api_key=secret_value_1)

In [18]:
py.plot(fig, filename = 'cap', auto_open=True)

'https://plotly.com/~crypto-financial-stability-monitor/6/'

fig = px.area(df_CapMrkt, x="time", y="CapMrktCurUSD", 
              title='Crypto Capitalization', 
              color="asset")
app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__=="__main__":
    app.run_server(mode='external')